In [3]:
import torch
import torch.nn as nn

class CIFAR10LightModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10LightModel, self).__init__()
        def dw_sep_conv(in_ch, out_ch, stride=1):
            return nn.Sequential(
                nn.Conv2d(in_ch, in_ch, 3, stride, 1, groups=in_ch, bias=False),
                nn.BatchNorm2d(in_ch),
                nn.GELU(),
                nn.Conv2d(in_ch, out_ch, 1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.GELU(),
            )

        def inverted_residual(in_ch, out_ch, expansion, stride):
            mid_ch = in_ch * expansion
            return nn.Sequential(
                nn.Conv2d(in_ch, mid_ch, 1, bias=False),
                nn.BatchNorm2d(mid_ch),
                nn.GELU(),
                dw_sep_conv(mid_ch, out_ch, stride),
            ) if stride > 1 or in_ch != out_ch else nn.Sequential(
                nn.Conv2d(in_ch, mid_ch, 1, bias=False),
                nn.BatchNorm2d(mid_ch),
                nn.GELU(),
                dw_sep_conv(mid_ch, out_ch, stride),
            )

        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.GELU(),
        )

        self.features = nn.Sequential(
            dw_sep_conv(32, 64, 1),
            inverted_residual(64, 128, 4, 2),
            inverted_residual(128, 128, 4, 1),
            inverted_residual(128, 256, 4, 2),
            dw_sep_conv(256, 512, 2),
        )

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [1]:
!sudo apt update
!sudo apt install nvidia-driver-470

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,224 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Pac

In [ ]:
#when using cpu
import torch
import torchvision
import torchvision.transforms as transforms
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_model():
    # Data Augmentation and Preprocessing
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    # Model, Loss, Optimizer, Scheduler
    model = CIFAR10LightModel().to('cpu')
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=3e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=50)

    # Training Loop
    for epoch in range(50):
        model.train()
        running_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for inputs, targets in pbar:
            inputs, targets = inputs.to('cpu'), targets.to('cpu')

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / len(train_loader))

        scheduler.step()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to('cpu'), targets.to('cpu')
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        acc = 100.0 * correct / total
        print(f"Validation Accuracy after Epoch {epoch+1}: {acc:.2f}%")

    # Save Model
    torch.save(model.state_dict(), "cifar10_light_model.pth")
    print("Model training complete. Saved as 'cifar10_light_model.pth'")

if __name__ == "__main__":
    train_model()

Files already downloaded and verified
Files already downloaded and verified


Epoch 1:   3%|▎         | 12/391 [01:06<32:50,  5.20s/it, loss=0.0686]

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_model():
    # Data Augmentation and Preprocessing
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    # Model, Loss, Optimizer, Scheduler
    model = CIFAR10LightModel().to('cuda')
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=3e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=50)

    # Training Loop
    for epoch in range(50):
        model.train()
        running_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for inputs, targets in pbar:
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / len(train_loader))

        scheduler.step()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to('cuda'), targets.to('cuda')
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        acc = 100.0 * correct / total
        print(f"Validation Accuracy after Epoch {epoch+1}: {acc:.2f}%")

    # Save Model
    torch.save(model.state_dict(), "cifar10_light_model.pth")
    print("Model training complete. Saved as 'cifar10_light_model.pth'")

if __name__ == "__main__":
    train_model()

Files already downloaded and verified
Files already downloaded and verified


Epoch 1: 100%|██████████| 391/391 [00:32<00:00, 12.18it/s, loss=1.37]


Validation Accuracy after Epoch 1: 60.34%


Epoch 2: 100%|██████████| 391/391 [00:29<00:00, 13.46it/s, loss=0.897]


Validation Accuracy after Epoch 2: 65.62%


...

Validation Accuracy after Epoch 48: 91.99%


Epoch 49: 100%|██████████| 391/391 [00:29<00:00, 13.04it/s, loss=0.0441]


Validation Accuracy after Epoch 49: 91.89%


Epoch 50: 100%|██████████| 391/391 [00:28<00:00, 13.56it/s, loss=0.045]


Validation Accuracy after Epoch 50: 91.90%
Model training complete. Saved as 'cifar10_light_model.pth'


In [7]:
from fvcore.nn import FlopCountAnalysis
model = CIFAR10LightModel()
inputs = torch.randn(1, 3, 32, 32)
flops = FlopCountAnalysis(model, inputs)
print(f"FLOPs: {flops.total() / 1e6:.2f}M")

FLOPs: 96.27M


In [8]:
torch.save(model.state_dict(), "cifar10_model.pth")
import os
print(f"Model Size: {os.path.getsize('cifar10_model.pth') / 1e6:.2f} MB")


Model Size: 2.21 MB


In [10]:
model = CIFAR10LightModel()  # Initialize your model

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Trainable Parameters: {total_params}")


Total Trainable Parameters: 537994


In [9]:
pip install torchsummary


In [6]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=f22ff1207e90fccef377419112f8adb6f097faea8a855d92d8f1473847fa8768
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=46221bfe925a670162ba2ee871abecd7f1061e513a0805fd46e84f16827ab128
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
